In [21]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file determines how the Lyman-alpha flux
# in the MagE apertures changes due to seeing effects,
# as estimated from the narrowband Lyman-alpha
# images.

In [22]:
import os
import glob

from datetime import datetime, timezone

import sigfig

from numba import njit

from sigfig import round as round_sigfig
from decimal import Decimal, getcontext

import numpy as np

from math import floor, log10

from scipy.interpolate import RegularGridInterpolator

from astropy.io import fits
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
from astropy import uncertainty as unc

import matplotlib.pyplot as plt

In [ ]:
def func():

    @njit
    def convolve_uncertainties(uncertainty, mask, stdv, psf_stdv, shape):

        '''
        Convolve the uncertainties of an image

        Parameters:
            uncertainty : numpy.ndarray
                The estimated uncertainty of the pixels in the image
            mask : numpy.ndarray
                The mask of the slit aperture
            stdv : numpy.float64
                The standard deviation of the Gaussian convolution kernel in pixels
            psf_stdv : numpy.float64
                The standard deviation of the image's PSF in pixels
            shape : tuple
                The dimensions of the image

        Returns:
            uncertainty_convolved : numpy.ndarray
                The convolved estimated uncertainty of the pixels in the image, 
                broadcasted to the shape of the image
        '''


        # Set the kernel size to 3 standard deviations of the 
        # width of the time-averaged seeing conditions
        kernel_size = int(3 * stdv)

        # Make a dummy array of the convolved uncertainties with the same shape as the image
        uncertainty_convolved = np.zeros_like(uncertainty, dtype=np.float64)

        #image_pixels = np.stack(np.indices((shape[0], shape[1])), axis=-1).reshape(-1,2)

        slit_pixels = np.vstack(np.where(mask > 0)).T

        '''
        # For each pixel column in the image
        for i in range(shape[0]):

            # For each pixel row in the image
            for j in range(shape[1]):
        '''

        for pixel in slit_pixels:

            '''
            # Set the squared sum of the weighted uncertainties to zero, to be added to in the double loop below
            squared_sum = 0.0

            # For each pixel within the kernel size
            #for k in range(-np.array([kernel_size,i]).min(), np.array([kernel_size+i,shape[0]-1]).min()):
            #    for l in range(-np.array([kernel_size,j]).min(), np.array([kernel_size+j,shape[1]-1]).min()):
            for k in range(np.array([0, pixel[0]-kernel_size], dtype=np.int64).max(), np.array([shape[0]-1, pixel[0]+kernel_size], dtype=np.int64).min() + 1):
                for l in range(np.array([0, pixel[1]-kernel_size], dtype=np.int64).max(), np.array([shape[1]-1, pixel[1]+kernel_size], dtype=np.int64).min() + 1):

                    # Calculate the weight of the pixel
                    weight = np.exp(-(abs(pixel[0]-k)**2 + abs(pixel[1]-l)**2) / (2 * stdv**2))
                    weight /= 2 * np.pi * stdv**2

                    # Add the weighted uncertainty to the total squared sum of uncertainties
                    squared_sum += (weight * uncertainty[k,l])**2

            uncertainty_convolved[pixel[0],pixel[1]] = np.sqrt(squared_sum)

            #print(uncertainty[i,j], uncertainty_convolved[i,j])
            '''

            #kernel_pixels = np.array([[x,y] for y in range(np.array([0, pixel[1]-kernel_size], dtype=np.int64).max(), np.array([shape[1]-1, pixel[1]+kernel_size], dtype=np.int64).min() + 1) 
            #    for x in range(np.array([0, pixel[0]-kernel_size], dtype=np.int64).max(), np.array([shape[0]-1, pixel[0]+kernel_size], dtype=np.int64).min() + 1)], dtype=np.int64)

            squared_sum = 0.0

            #for i in range(shape[0]):
            #for j in range(shape[1]):

            #for pixel_1 in image_pixels:
            #    for pixel_2 in image_pixels:
            for k in range(np.array([0, pixel[0]-kernel_size], dtype=np.int64).max(), np.array([shape[0]-1, pixel[0]+kernel_size], dtype=np.int64).min() + 1):
                for l in range(np.array([0, pixel[1]-kernel_size], dtype=np.int64).max(), np.array([shape[1]-1, pixel[1]+kernel_size], dtype=np.int64).min() + 1):

            #for pixel_1 in kernel_pixels:
            #    for pixel_2 in kernel_pixels:

                    #midpoint = (pixel_1 + pixel_2) / 2

                    #squared_sum = np.exp(-np.dot(pixel - pixel_1, pixel - pixel_1) / (2 * stdv**2) - np.dot(pixel - pixel_2, pixel - pixel_2) / (2 * stdv**2) - np.dot(pixel_1 - pixel_2, pixel_1 - pixel_2) / (4 * psf_stdv**2)) * uncertainty_interp((pixel_1 + pixel_2) / 2)**2
                    #squared_sum = np.exp(-np.dot((pixel - pixel_1).astype(np.float64), (pixel - pixel_1).astype(np.float64)) / (2 * stdv**2) - np.dot((pixel - pixel_2).astype(np.float64), (pixel - pixel_2).astype(np.float64)) / (2 * stdv**2) - np.dot((pixel_1 - pixel_2).astype(np.float64), (pixel_1 - pixel_2).astype(np.float64)) / (4 * psf_stdv**2)) * uncertainty[round(midpoint[0]), round(midpoint[1])]**2

                    squared_sum += np.exp(-np.dot((np.array([k,l], dtype=np.int64) - pixel).astype(np.float64), (np.array([k,l], dtype=np.int64) - pixel).astype(np.float64)) / stdv**2) * uncertainty[k,l]**2

            variance = squared_sum * (2 * np.sqrt(np.pi) * stdv * psf_stdv / np.sqrt(stdv**2 + psf_stdv**2))**2 / (2 * np.pi * stdv**2)**2

            uncertainty_convolved[pixel[0],pixel[1]] = np.sqrt(variance)

        # Multiply the dummy array of the convolved uncertainties by the convolved uncertainty
        #uncertainty_convolved = uncertainty_convolved * np.sqrt(squared_sum)

        return uncertainty_convolved

    # Establish common directories
    home = os.getcwd()
    #data = f'{home}/data'
    results = f'{home}/results'
    
    # Array of the slit IDs
    #slits = np.array(['M5', 'M4', 'M6', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9'], 
    #    dtype=str)
    
    # Dictionary of roughly-estimated time-weighted seeing conditions, 
    # written as the FWHM of the PSF in arcseconds
    '''
    seeing = {
        'M5' : 0.97,
        'M4' : 0.71,
        'M6' : 0.76,
        'M3' : 0.70,
        'M0' : 1.34,
        'M2' : 0.77,
        'M7' : 0.73,
        'M8' : 0.70,
        'M9' : 0.68
    }
    '''

    # Dictionary of the MagE slit aperture IDs, listing, from left to right: the FWHM of the seeing conditions 
    # in arcseconds, airmasses of each exposure, written as sec(z), and the exposure times of each exposure
    slits = {
        'M5' : [0.97, [1.756, 1.668, 1.602, 1.553, 1.531], [2700, 2700, 2700, 2700, 2700]],
        'M4' : [0.71, [1.529, 1.532, 1.549], [2400, 2400, 2400]],
        'M6' : [0.76, [1.530, 1.549, 1.581, 1.531, 1.552], [2700, 2400, 2400, 2700, 2700]],
        'M3' : [0.70, [1.775, 1.690, 1.627, 1.571, 1.542], [2400, 2400, 2400, 2400, 2400]],
        'M0' : [1.34, [1.540, 1.528, 1.536, 1.565, 1.656], [2700, 2700, 2700, 2700, 2700]],
        'M2' : [0.77, [1.584, 1.639, 1.592], [2700, 2700, 2700]],
        'M7' : [0.73, [1.563, 1.600, 1.665, 1.569, 1.610, 1.672], [2400, 2400, 2400, 2400, 2400, 2400]],
        'M8' : [0.70, [1.528, 1.537, 1.747, 1.529, 1.539, 1.757], [2400, 2400, 2400, 2400, 2400, 2400]],
        'M9' : [0.68, [1.738, 1.654, 1.592, 1.552, 1.531], [2700, 2700, 2700, 2700, 2700]]
    }

    '''
    # Dictionary of the airmasses of the individual slit pointings,
    # written as sec(z)
    airmass = {
        'M5' : [1.756, 1.668, 1.602, 1.553, 1.531],
        'M4' : [1.529, 1.532, 1.549],
        'M6' : [1.530, 1.549, 1.581, 1.531, 1.552],
        'M3' : [1.775, 1.690, 1.627, 1.571, 1.542],
        'M0' : [1.540, 1.528, 1.536, 1.565, 1.656],
        'M2' : [1.584, 1.639, 1.592],
        'M7' : [1.563, 1.600, 1.665, 1.569, 1.610, 1.672],
        'M8' : [1.528, 1.537, 1.747, 1.529, 1.539, 1.757],
        'M9' : [1.738, 1.654, 1.592, 1.552, 1.531]
    }

    # Dictionary of the exposure times of the individual slit pointings,
    # written in seconds
    exposures = {
        'M5' : [2700, 2700, 2700, 2700, 2700],
        'M4' : [2400, 2400, 2400],
        'M6' : [2700, 2400, 2400, 2700, 2700],
        'M3' : [2400, 2400, 2400, 2400, 2400],
        'M0' : [2700, 2700, 2700, 2700, 2700],
        'M2' : [2700, 2700, 2700],
        'M7' : [2400, 2400, 2400, 2400, 2400, 2400],
        'M8' : [2400, 2400, 2400, 2400, 2400, 2400],
        'M9' : [2700, 2700, 2700, 2700, 2700]       
    }
    '''

    #lya_files = glob.glob(f'{data}/hst/Lya_contsub*.fits')
    
    # HST filter IDs used as continuum estimates for the narrowband
    # Lya maps
    filters = np.array(['F390W', 'F555W'], dtype=str)

    # File path to the mask of the two largest arcs of the Sunburst Arc
    # in the v5 HST data reduction WCS
    arc_mask_file = f'{results}/masks/arc_mask_v5.fits'

    # Get a cutout of the arc mask. This is necessary because we will
    # convolve the Lya maps later, which is computationally expensive for
    # a very large footprint.
    arc_mask = fits.getdata(arc_mask_file)[4000:5600, 4400:5700]

    # Invert the arc mask so that it is 0 where the arc is and
    # 1 elsewhere. This will make it easier to create an effective
    # mask of a slit, except where the arc is.
    inverted_arc_mask = np.where(arc_mask > 0, 0, 1)

    #with open('seeing_simulation.txt', 'a') as table:

    #table = ''

    # Make empty lists to append the results of the simulation to
    fluxes, n_fluxes, fluxes_convolved, n_fluxes_convolved, ratios, n_ratios = [], [], [], [], [], []

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Get the FWHM of the seeing conditions and the airmasses and exposure times of each exposure
        seeing, airmass, exposures = slits[slit_id][0], slits[slit_id][1], slits[slit_id][2]

        #table = table + f'{slit_id} & '
    
        # File path to the slit mask
        slit_mask_file = f'{results}/masks/{slit_id}_mask_v5.fits'

        # Get a cutout of the mask
        slit_mask = fits.getdata(slit_mask_file)[4000:5600, 4400:5700]

        # For each filter used to estimate the continuum
        for j, filter in enumerate(filters):

            # File path to the corresponding Lya map
            file = f'{results}/lya_maps/Lya_cont_sub_{filter}.fits'

            # Get cutouts of the map and the uncertainty map
            lya = fits.open(file)[0].data[4000:5600, 4400:5700]
            #mask = fits.open(f'{results}/box_for_median_imcoords_mask_v5.fits')[0].data[4300:5300, 4700:5400]
            n_lya = fits.open(file)[1].data[4000:5600, 4400:5700]
            
            #n_lya_interp = RegularGridInterpolator((range(np.shape(lya)[0]), range(np.shape(lya)[1])), n_lya)
            
            # We are going to sum the flux in the Lya map
            # in the slit, so determine the associated noise
            #n_lya = np.sqrt(np.sum(np.square(n_lya * slit_mask)))

            #stddev = ((0.70 * (1.641)**0.6) / 2.355) / 0.03

            # The standard deviation of the exposure time-weighted PSF in pixels
            stdv = ((seeing * (np.sum(np.multiply(airmass, exposures) / np.sum(exposures)))**0.6) / 2.355) / 0.03

            # A Gaussian kernel with the above standard deviation
            kernel = Gaussian2DKernel(x_stddev=stdv, y_stddev=stdv)

            # Convolve the Lya map with the Gaussian kernel representing the exposure time-weighted seeing conditions
            lya_convolved = convolve_fft(lya, kernel)
            
            # Compute the error of each pixel in the convolved Lya map as the
            # standard deviation of the pixel values outside of the arc mask but
            # inside the slit mask
            #n_convolved = np.std(lya_convolved * slit_mask * inverted_arc_mask)
            #n_convolved = np.ones_like(lya_convolved) * n_convolved
            #n_convolved = np.sqrt(np.sum(np.square(n_convolved * slit_mask)))

            # Convolve the uncertainties of the Lya map with the Gaussian kernel 
            # representing the exposure time-weighted seeing conditions
            n_lya_convolved = convolve_uncertainties(n_lya.astype(np.float64), (slit_mask * arc_mask).astype(np.int64), stdv.astype(np.float64), 0.067 / 0.03, np.array([np.shape(lya)[0], np.shape(lya)[1]], dtype=np.int64))

            # Sum the unconvolved Lya flux in the slit
            flux = np.sum(slit_mask * lya)
            fluxes.append(flux)

            # Calculate the propagated uncertainty of the unconvolved Lya flux in the slit
            n_flux = np.sqrt(np.sum((slit_mask * n_lya)**2))
            n_fluxes.append(n_flux)

            # Sum the convolved Lya flux in the slit
            flux_convolved = np.sum(slit_mask * lya_convolved)
            fluxes_convolved.append(flux_convolved)

            # Calculate the uncertainty of the convolved flux
            n_flux_convolved = np.sqrt(np.sum((slit_mask * n_lya_convolved)**2))
            n_fluxes_convolved.append(n_flux_convolved)

            # Calculate the ratio between the convolved and unconvolved flux
            ratio = flux_convolved / flux
            ratios.append(ratio)

            # Calculate the propagated uncertainty of the ratio between the convolved and unconvolved Lya flux
            n_ratio = np.absolute(ratio) * np.sqrt((n_flux / flux)**2 + (n_flux_convolved / flux_convolved)**2)
            n_ratios.append(n_ratio)

            #n = str(round_sigfig(n, sigfigs=1))
            #n_convolved = str(round_sigfig(n_convolved, sigfigs=1))
            #ratio_uncertainty = str(round_sigfig(ratio_uncertainty, sigfigs=1))

            #place = -int(floor(log10(abs(float(ratio_uncertainty)))) + 1) if float(ratio_uncertainty) >= 1 else -int(floor(log10(abs(float(ratio_uncertainty)))))

            #ratio = round(ratio, -int(floor(log10(abs(float(ratio_uncertainty))))) + 1)

            #ratio_uncertainty = ratio_uncertainty.split('.')[1].count('0', )

            #ratio = round_sigfig(ratio, decimals=ratio_uncertainty.split('.')[1].count('0') + 1)

            #line.join([line, f'${ratio}\pm {ratio_uncertainty}$'])
            #table = table +  f'${flux}\pm {n} & {flux_convolved}\pm {n_convolved} & {ratio}\pm {ratio_uncertainty}$'

            #table = table + ' & ' if j==0 else table

            #np.savetxt('seeing_simulation.txt')

        #table = table + ' \\\\ \n' if i < 8 else ''

        #table.write(line)
    
    header = 'The Lyα fluxes and ratios measured in the seeing effects simulation\n' \
        + f'Created by seeing.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}\n' \
        + '\n' \
        + 'The columns below are organized as follows. There are three types of measurements: unconvolved and convolved Lyα fluxes in a slit aperture (e/s),\n' \
        + 'and the ratio between the two fluxes. Each measurement type has a set of 4 columns, arranged consecutively in the order mentioned previously.\n' \
        + 'Those sets of 4 columns each have 2 consecutive subsets of 2 columns. In each subset, the first two columns represent, respectively, the\n' \
        + 'measurement and uncertainty from the F390W-based Lyα map, and the last two columns represent, respectively, the measurement and uncertainty\n' \
        + 'from the F555W-based Lyα map. The rows corresponding to the different MagE slit apertures are arranged, from top to bottom, in the following\n' \
        + 'order: M5, M4, M6, M3, M0, M2, M7, M8, and M9.\n' \
    
    np.savetxt(f'{results}/seeing_simulation_measurements_results.txt', 
        np.array([fluxes[::2], n_fluxes[::2], fluxes[1::2], n_fluxes[1::2], fluxes_convolved[::2], n_fluxes_convolved[::2], fluxes_convolved[1::2], n_fluxes_convolved[1::2], ratios[::2], n_ratios[::2], ratios[1::2], n_ratios[1::2]], dtype=np.float64).T, 
        delimiter=' ', header=header, encoding='utf-8')

def tabulate():

    def round_to_uncertainties(measurement, uncertainty):

        '''
        Round the uncertainty to one significant figure and the measurement to
        the same significant digit as the uncertainty

        Parameters:

            measurement : numpy.float64
                Value of the measurement

            uncertainty : numpy.float64
                Estimated uncertainty of the measurement

        Returns:

            measurement : str
                Value of the measurement, rounded to the same significant digit as
                the 1 significant figure-rounded uncertainty

            uncertainty : str
                Estimated uncertainty of the measurement, rounded to 1 significant figure
        '''

        # Round the uncertainty to one significant figure
        uncertainty = sigfig.round(uncertainty, sigfigs=1, type=str)

        # If the uncertainty is less than 1
        if '.' in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, decimals=len(uncertainty.split('.')[1]), type=str)

        # If the uncertainty is greater than 1
        elif '.' not in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, len(str(measurement).replace('-','').split('.')[0]) - len(uncertainty) + 1, type=str)

        return measurement, uncertainty

    slits = {
        'M5' : [],
        'M4' : [],
        'M6' : [],
        'M3' : [],
        'M0' : [],
        'M2' : [],
        'M7' : [],
        'M8' : [],
        'M9' : []
    }

    home = os.getcwd()
    results = f'{home}/results'

    table = '\\begin{deluxetable*}{r|lll|lll}\n' \
        + '\n' \
        + '\\tablecaption{Simulated aperture flux changes due to atmospheric seeing \label{tab:seeing_simulation}}\n' \
        + '\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{Slit} & \n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{F390W} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{F555W} & \n' \
        + '\t\colhead{}\n' \
        + '\t\\\\\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{Unconvolved} &\n' \
        + '\t\colhead{Convolved} &\n' \
        + '\t\colhead{Ratio} & \n' \
        + '\t\colhead{Unconvolved} &\n' \
        + '\t\colhead{Convolved} &\n' \
        + '\t\colhead{Ratio}\n' \
        + '\t\\\\\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{}\n' \
        + '}\n' \
        + '\n' \
        + '\startdata\n' \
        + '\n'

    lya_measurements = np.loadtxt(f'{results}/seeing_simulation_measurements_results.txt', delimiter=' ', comments='#')

    for i, slit_id in enumerate(slits):

        slit_lya_measurements = lya_measurements[i]

        f390w_flux, n_f390w_flux = round_to_uncertainties(slit_lya_measurements[0], slit_lya_measurements[1])
        f390w_flux_convolved, n_f390w_flux_convolved = round_to_uncertainties(slit_lya_measurements[4], slit_lya_measurements[5])
        f390w_ratio, n_f390w_ratio = round_to_uncertainties(slit_lya_measurements[8], slit_lya_measurements[9])

        f555w_flux, n_f555w_flux = round_to_uncertainties(slit_lya_measurements[2], slit_lya_measurements[3])
        f555w_flux_convolved, n_f555w_flux_convolved = round_to_uncertainties(slit_lya_measurements[6], slit_lya_measurements[7])
        f555w_ratio, n_f555w_ratio = round_to_uncertainties(slit_lya_measurements[10], slit_lya_measurements[11])

        # Add a separating horizontal line between the LyC-leaking and non-LyC-leaking spectra
        table = table + '\hline\n' if slit_id == 'M0' else table + ''

        table = table + f'{slit_id} & ${f390w_flux}\pm{n_f390w_flux}$ & ${f390w_flux_convolved}\pm{n_f390w_flux_convolved}$ & ${f390w_ratio}\pm{n_f390w_ratio}$ ' \
            + f'& ${f555w_flux}\pm{n_f555w_flux}$ & ${f555w_flux_convolved}\pm{n_f555w_flux_convolved}$ & ${f555w_ratio}\pm{n_f555w_ratio}$'

        table = table + ' \\\\\n' if i != len(slits) - 1 else table + '\n'

    table = table + '\n\enddata\n' \
        + '\n' \
        + '\\tablecomments{From left to right: slit, flux inside the slit of the unconvolved and simulated seeing-convolved F390W-based Ly$\\alpha$ map, the ratio between the two fluxes, and likewise for the F555W-based Ly$\\alpha$ map. Ratios $>$ 1 indicate the flux in the aperture increased after the convolution, and ratios $<$ 1 indicate the flux in the aperture decreased after the convolution. The convolution used a 2-dimensional Gaussian kernel of the combined effect of the time-weighted seeing conditions and airmasses (\S\,\\ref{sssec:disc_lya_seeing}).}\n' \
        + '\n' \
        + '\end{deluxetable*}'

    # Save the table
    f = open(f'{results}/tables/seeing_simulation_measurements_table.txt', 'w', encoding='utf-8')
    f.write(table)
    f.close()

In [ ]:
func()

In [ ]:
tabulate()